In [1]:
import orekit
from orekit.pyhelpers import setup_orekit_curdir, download_orekit_data_curdir

download_orekit_data_curdir()
orekit.initVM()
setup_orekit_curdir()

In [2]:
from org.orekit.propagation.analytical import BrouwerLyddanePropagator, KeplerianPropagator, EcksteinHechlerPropagator
from org.orekit.propagation import PropagationType
from org.orekit.orbits import KeplerianOrbit, PositionAngle, EquinoctialOrbit
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.bodies import CelestialBodyFactory
from org.orekit.utils import Constants, PVCoordinates
from org.orekit.frames import FramesFactory
from org.orekit.attitudes import NadirPointing
from org.orekit.bodies import OneAxisEllipsoid
from org.orekit.models.earth import ReferenceEllipsoid
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.orbits import OrbitType
import matplotlib.pyplot as plt
from org.orekit.forces.gravity.potential import GravityFieldFactory

import math

In [9]:
# create a keplerian orbit for a satellite at 400km, 600km, and 800km altitude
# initialDate = AbsoluteDate(2023, 1, 1, 12, 0, 0.0, TimeScalesFactory.getUTC())
mu = Constants.WGS84_EARTH_MU
inertialFrame = FramesFactory.getEME2000()
# a, e, i, w, W, v = 6771.7e3, 0.0007, math.radians(51.64), math.radians(13.8511), math.radians(21.3671), math.radians(98.5566)
# ex = e*math.cos(w + W)
# ey = e*math.sin(w + W)
# hx = math.tan(i/2) * math.cos(W)
# hy = math.tan(i/2)*math.sin(W)
# lv = v + w + W

# orbits = [EquinoctialOrbit(a, ex, ey, hx, hy, lv, inertialFrame, initialDate, mu)]

# # Use NadirPointing as the AttitudeProvider (points towards Earth center)
body = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                        Constants.WGS84_EARTH_FLATTENING, 
                        inertialFrame)
attitudeProvider = NadirPointing(inertialFrame, body)
initDate = AbsoluteDate.J2000_EPOCH.shiftedBy(584.)
position = Vector3D(3220103., 69623., 6149822.)
velocity = Vector3D(6414.7, -2006., -3180.)

initialOrbit = EquinoctialOrbit(PVCoordinates(position, velocity), 
                                FramesFactory.getEME2000(), initDate, mu)

# Brouwer-Lyddane Propagator parameters
mass = 100.0  # arbitrary satellite mass in kg
referenceRadius = Constants.WGS84_EARTH_EQUATORIAL_RADIUS
c20 = -1.08e-3  # un-normalized zonal coefficient
c30 = 2.53e-6   # un-normalized zonal coefficient
c40 = 1.62e-6   # un-normalized zonal coefficient
c50 = 2.28e-7   # un-normalized zonal coefficient
c60 = -5.41e-7  # un-normalized zonal coefficient
M2 = 1.0e-14 # ignoring empirical drag

days = 1000
dates = []

for day in range(1, days):
    finalDate = initDate.shiftedBy(86400.0 * day)
    dates.append(finalDate.toString())

plt.figure(figsize=(12, 6))

# Propagate each satellite and plot its altitude
altitudes = []
extrapolator = BrouwerLyddanePropagator(initialOrbit, GravityFieldFactory.getUnnormalizedProvider(GravityFieldFactory.getNormalizedProvider(5, 0)),
                                                BrouwerLyddanePropagator.M2)

for day in range(1, days):
    try: 
        propagatedOrbit = extrapolator.propagateOrbit(initDate.shiftedBy(86400.0 * day))
        altitude = propagatedOrbit.getA() - Constants.WGS84_EARTH_EQUATORIAL_RADIUS
        altitudes.append(altitude/1000)  # in km
    except:
        altitudes.append(None)  # if propagation failed
plt.plot(dates, altitudes, label=f'Altitude: {initialOrbit.getA()/1e3 - 6378.0} km')

plt.xticks(dates[::30])  # display every 30th date for readability
plt.xlabel('Date')
plt.ylabel('Altitude (km)')
plt.title('Altitude over Time for Various Satellites')
plt.grid(True)
plt.tight_layout()
# plt.legend()
plt.ylim(0, 1000)
plt.xticks(rotation=45)
plt.show()

AttributeError: 'KeplerianOrbit' object has no attribute 'getOrbit'

<Figure size 1200x600 with 0 Axes>

In [ ]:
import numpy as np

def kepler_prop(jd_start,jd_stop,step_size,a,e,i,w,W,V):
    """Analytical Propagation. This function calculates the orbit of a
    satellite around the Earth assuming two body-dynamics based
    on Keplerian elements and step size.
    Args:
        jd_start:  starting time in Julian Date
        jd_stop: ending time in Julian Date
        step_size (int): sampling time of the orbit in seconds
        a (float): semi-major axis in Kilometers
        e (float): eccentricity of the orbit
        i (float): inclination of the orbit in degrees
        w (float): argument of periapsis in degrees
        W (float): longitude of the ascending node in degrees
        V (float): true anomaly in degrees
    Returns:
        ephemeris (list): list in the form [(time, position, velocity), (time, position, velocity), ...]
    """
    # Convert angles to radians
    i, w, W, V = map(np.deg2rad, [i, w, W, V])

    # Defining radial distance and semi-latus rectum
    p = a * (1 - (e**2))
    r = p / (1 + e * np.cos(V))

    # empty list to store the time step in each iteration
    ephemeris = []

    # calculate the number of steps between jd_start and jd_stop if the step size is step_size seconds) 
    t_diff = jd_stop-jd_start  # time difference in Julian Days
    t_diff_secs = 86400 * t_diff
    current_jd = jd_start
    steps = math.ceil(t_diff_secs/step_size)  # total number of integer steps
    for step in range(steps):
        # Compute the mean motion
        n = np.sqrt(GM_earth / (a**3))

        # Compute the eccentric anomaly at t=t0
        cos_Eo = ((r * np.cos(V)) / a) + e
        sin_Eo = (r * np.sin(V)) / (a * np.sqrt(1 - e**2))
        Eo = np.arctan2(sin_Eo, cos_Eo)
        Eo = Eo if Eo >= 0 else Eo + 2*np.pi

        # Compute mean anomaly at start point
        Mo = Eo - e * np.sin(Eo)

        # Compute the mean anomaly at t+newdt
        Mi = Mo + n * (step * step_size)

        # Solve Kepler's equation to compute the eccentric anomaly at t+newdt
        M = Mi

        # Initial Guess at Eccentric Anomaly
        if M < np.pi:
            E = M + (e / 2)
        if M >= np.pi:
            E = M - (e / 2)

        # Iterative solution for E
        r_tol = 1e-7  # relative tolerance
        f = E - e * np.sin(E) - M
        f_prime = 1 - e * np.cos(E)
        ratio = f / f_prime
        while abs(ratio) > r_tol:
            f = E - e * np.sin(E) - M
            f_prime = 1 - e * np.cos(E)
            ratio = f / f_prime
            E -= ratio

        Ei = E

        # Compute the gaussian vector component x,y
        x_new = a * (np.cos(Ei) - e)
        y_new = a * (np.sqrt(1 - e**2) * np.sin(Ei))
        # Compute the in-orbital plane Gaussian Vectors
        # This gives P and Q in ECI components
        P = np.array(
            [
                [np.cos(W) * np.cos(w) - np.sin(W) * np.cos(i) * np.sin(w)],
                [np.sin(W) * np.cos(w) + np.cos(W) * np.cos(i) * np.sin(w)],
                [np.sin(i) * np.sin(w)],
            ]
        )
        Q = np.array(
            [
                [-np.cos(W) * np.sin(w) - np.sin(W) * np.cos(i) * np.cos(w)],
                [-np.sin(W) * np.sin(w) + np.cos(W) * np.cos(i) * np.cos(w)],
                [np.sin(i) * np.cos(w)],
            ]
        )

        # Compute the position vector at t+newdt
        # We know the inertial vector components along the P and Q vectors.
        # Thus we can project the satellite position onto the ECI basis.
        # calcualting the new x coordiante

        cart_pos_x_new = (x_new * P[0, 0]) + (y_new * Q[0, 0])
        cart_pos_y_new = (x_new * P[1, 0]) + (y_new * Q[1, 0])
        cart_pos_z_new = (x_new * P[2, 0]) + (y_new * Q[2, 0])
        
        # Compute the range at t+dt
        r_new = a * (1 - e * (np.cos(Ei)))
        # Compute the gaussian velocity components
        cos_Ei = (x_new / a) + e
        sin_Ei = y_new / a * np.sqrt(1 - e**2)
        f_new = (np.sqrt(a * GM_earth)) / r_new
        g_new = np.sqrt(1 - e**2)
       
        cart_vel_x_new = (-f_new * sin_Ei * P[0, 0]) + (f_new * g_new * cos_Ei * Q[0, 0])
        cart_vel_y_new = (-f_new * sin_Ei * P[1, 0]) + (f_new * g_new * cos_Ei * Q[1, 0])
        cart_vel_z_new = (-f_new * sin_Ei * P[2, 0]) + (f_new * g_new * cos_Ei * Q[2, 0])

        pos = ([cart_pos_x_new, cart_pos_y_new, cart_pos_z_new])
        vel = ([cart_vel_x_new, cart_vel_y_new, cart_vel_z_new])
        
        ephemeris.append([current_jd, pos, vel])
        # Update the JD time stamp
        current_jd = current_jd + step_size/86400.0  # convert seconds to days
    return ephemeris   
